In [1]:
import pandas as pd
import numpy as np 
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

In [2]:
data1 = pd.read_csv('../stats_100_etfs.csv')
data = pd.read_csv('../hun_etfs.csv')
data

,date,open,high,low,close,volume,ticker
0,2015-01-05,204.17,204.37,201.35,201.72,169632646,SPY
1,2015-01-06,202.09,202.72,198.86,199.82,209151408,SPY
2,2015-01-07,201.42,202.72,200.88,202.31,125346709,SPY
3,2015-01-08,204.01,206.16,203.99,205.90,147217784,SPY
4,2015-01-09,206.40,206.42,203.51,204.25,158567288,SPY
...,...,...,...,...,...,...,...
125695,2019-12-24,22.82,22.82,22.77,22.78,2633089,BKLN
125696,2019-12-26,22.79,22.81,22.79,22.79,2999603,BKLN
125697,2019-12-27,22.79,22.80,22.78,22.79,2654723,BKLN
125698,2019-12-30,22.80,22.81,22.71,22.71,3697412,BKLN


In [5]:
df = data.loc[data['ticker'] == 'SPY']
df = df.set_index('date')
df

,open,high,low,close,volume,ticker
date,,,,,,
2015-01-05,204.17,204.37,201.35,201.72,169632646,SPY
2015-01-06,202.09,202.72,198.86,199.82,209151408,SPY
2015-01-07,201.42,202.72,200.88,202.31,125346709,SPY
2015-01-08,204.01,206.16,203.99,205.90,147217784,SPY
2015-01-09,206.40,206.42,203.51,204.25,158567288,SPY
...,...,...,...,...,...,...
2019-12-24,321.47,321.52,320.90,321.23,20270007,SPY
2019-12-26,321.65,322.95,321.64,322.94,31024188,SPY
2019-12-27,323.74,323.80,322.28,322.86,42554820,SPY


In [7]:
df = df[['close']]
#Take a look at the new data
print(df.head())

             close
date              
2015-01-05  201.72
2015-01-06  199.82
2015-01-07  202.31
2015-01-08  205.90
2015-01-09  204.25


In [20]:
# A variable for predicting 'n' days out into the future
forecast_out = 30 #'n=30' days
#Create another column (the target or dependent variable) shifted 'n' units up
df['Prediction'] = df[['close']].shift(-forecast_out)
#print the new data set
df

C:\Users\jayar\anaconda\Anaconda3\envs\PandaSpace\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,close,Prediction
date,,
2015-01-05,201.72,210.13
2015-01-06,199.82,209.98
2015-01-07,202.31,211.24
2015-01-08,205.90,211.21
2015-01-09,204.25,211.81
...,...,...
2019-12-24,321.23,NaN
2019-12-26,322.94,NaN
2019-12-27,322.86,NaN


In [9]:
### Create the independent data set (X)  #######
# Convert the dataframe to a numpy array
X = np.array(df.drop(['Prediction'],1))

#Remove the last 'n' rows
X = X[:-forecast_out]
print(X)

[[201.72]
 [199.82]
 [202.31]
 ...
 [309.1 ]
 [309.55]
 [311.79]]


In [11]:
### Create the dependent data set (y)  #####
# Convert the dataframe to a numpy array (All of the values including the NaN's)
y = np.array(df['Prediction'])
# Get all of the y values except the last 'n' rows
y = y[:-forecast_out]
print(y)

[210.13 209.98 211.24 ... 322.86 321.08 321.86]


In [13]:
# Split the data into 80% training and 20% testing
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [14]:
# Create and train the Support Vector Machine (Regressor)
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
svr_rbf.fit(x_train, y_train)

SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [15]:
# Testing Model: Score returns the coefficient of determination R^2 of the prediction. 
# The best possible score is 1.0
svm_confidence = svr_rbf.score(x_test, y_test)
print("svm confidence: ", svm_confidence)

svm confidence:  0.9580259094792138


In [16]:
# Create and train the Linear Regression  Model
lr = LinearRegression()
# Train the model
lr.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [17]:
# Testing Model: Score returns the coefficient of determination R^2 of the prediction. 
# The best possible score is 1.0
lr_confidence = lr.score(x_test, y_test)
print("lr confidence: ", lr_confidence)

lr confidence:  0.9469915084098509


In [18]:
# Set x_forecast equal to the last 30 rows of the original data set from Adj. Close column
x_forecast = np.array(df.drop(['Prediction'],1))[-forecast_out:]
print(x_forecast)

[[312.02]
 [311.93]
 [310.77]
 [310.27]
 [310.96]
 [313.37]
 [314.08]
 [315.48]
 [314.31]
 [311.64]
 [309.55]
 [311.46]
 [312.02]
 [314.87]
 [313.88]
 [313.53]
 [314.42]
 [317.13]
 [317.32]
 [319.5 ]
 [319.57]
 [319.59]
 [320.9 ]
 [320.73]
 [321.22]
 [321.23]
 [322.94]
 [322.86]
 [321.08]
 [321.86]]


In [19]:
# Print linear regression model predictions for the next 'n' days
lr_prediction = lr.predict(x_forecast)
print(lr_prediction)

# Print support vector regressor model predictions for the next 'n' days
svm_prediction = svr_rbf.predict(x_forecast)
print(svm_prediction)

[314.56684121 314.477081   313.32017164 312.82150382 313.50966542
 315.91324433 316.62135264 318.01762255 316.85073984 314.18785366
 312.10342215 314.00833324 314.56684121 317.40924781 316.42188551
 316.07281804 316.96044676 319.66322637 319.85272015 322.02691186
 322.09672536 322.11667207 323.42318177 323.25363471 323.74232918
 323.75230254 325.4577465  325.37795965 323.60270219 324.380624  ]
[321.59302984 321.68679655 321.11991891 320.95562057 321.26178248
 313.1776521  302.84098664 278.45142122 298.89724446 321.75775217
 321.17956406 321.67550671 321.59302984 288.86947866 306.08107052
 311.15988549 296.95376246 260.02879549 258.903659   254.00524179
 253.97307959 253.96444981 253.71803819 253.72910454 253.70418113
 253.70386017 253.68738645 253.68748565 253.70931809 253.69188569]
